In [ ]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np

# Parsing the wikipedia page

In [ ]:
# define our URL link string variable with '' ("" does not work !)
myurl='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [ ]:
#Parse the page with BS
r = requests.get(myurl)
#transform the URL into a Beautiful Soup object
soup = soup(r.text, 'lxml')

#find the table and extract the header wich is INDEX=0
header = soup.find(class_="wikitable sortable").find_all('tr')[0] #index=[0]
header_list = [i.text for i in header.find_all('th')]
header_list[2]=header_list[2].replace('\n', '')

header_list

In [ ]:
#find the table and extract the rows which start at INDEX=1
rows = soup.find(class_="wikitable sortable").find_all('tr')[1:] #index= [1:]
new_list=[]

for row in rows:
    cell = [i.text for i in row.find_all('td')]
    new_list.append(cell)

new_list

# Define the dataframe

In [ ]:
#defnie our dataframe and show it
df=pd.DataFrame(new_list,columns=header_list)

# Clean the dataframe and present it as requested

In [ ]:
#remove the remaining '\n' in some cells
df=df.replace('\n', '', regex=True)
#drop lines where borough is empty
df=df[df.Borough != 'Not assigned']


In [ ]:
#replace not assigned values in Neigbourhood by the Borough value
for line in range(0,df.shape[0]-1):
    if 'Not assigned' in df.iloc[line,2]:
        df.iloc[line,2]=df.iloc[line,1]

In [ ]:
# group the POSTCODES and separate neighbourhood by commas
df_grouped=df.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : ','.join(x))
df_grouped


In [ ]:
df_grouped.shape

# Get the latitude and longitude of each Postalcode using Geocode

In [ ]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None
# loop until you get the coordinates
#while(lat_lng_coords is None):
    #location = 'MG5' + ', Toronto, Ontario'
    #g = geocoder.google(location)
    #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

# Get the latitude and logitude of each Postalcode using the CSV file


In [ ]:
#import the coordinated data
coordinates=pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
coordinates

In [ ]:
# join both tables using "join" method and using the postcode as primary key
df_final=df_grouped.set_index('Postcode').join(coordinates.set_index('Postal Code'))
df_final

# Perform Analysis

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Get venues around our neighborhoods

In [ ]:
CLIENT_ID = 'HTQR2OI0SF52CJFAI1OPOE0UJAJO4NMJAFEFURZLYX031MKW' # your Foursquare ID
CLIENT_SECRET = 'X1VSW0ZPALP4ACGQO5C3TER5Y3HU12ZXFE5IUKM5DZCWQBWQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT= 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=df_final['Neighbourhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )


In [ ]:
toronto_venues

### Turn our Venues types into columns in order to prepare the data for clustering

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### Perfrom Clustering

In [ ]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


In [ ]:
neighborhoods_venues_sorted

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = neighborhoods_venues_sorted.join(toronto_merged.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!


# Show the results

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters